In [29]:
import yfinance as yf
import pandas as pd
import numpy as np
import os
import pickle
import joblib
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)

class CandleFit:
    def __init__(self, ticker: str, period: str = '5y'):
        """
        Initializes the CandleFit object with a specified ticker and period.

        Parameters:
        ticker (str): The ticker symbol of the stock.
        period (str): The period for which to download historical data.
        """
        self.ticker = ticker
        self.period = period
        self.data = self.get_ticker()

    def get_ticker(self):
        """
        Downloads historical data for the specified ticker and period.

        Returns:
        pd.DataFrame: A DataFrame containing the historical data with formatted dates and column names.
        """
        try:
            ticker_obj = yf.Ticker(self.ticker)
            hist = ticker_obj.history(period=self.period)
            hist.index = pd.to_datetime(hist.index).strftime('%Y-%m-%d')
            hist.columns = [col.lower() for col in hist.columns]
            return hist
        except Exception as e:
            print(f"Error downloading aux: {e}")
            return pd.DataFrame()
    
    @staticmethod
    def load_dict(key: str) -> dict:
        """
        Loads a dictionary from a pickle file based on the provided key.

        Parameters:
        key (str): The key to search for in the dictionary.

        Returns:
        dict: The dictionary associated with the provided key.

        Raises:
        FileNotFoundError: If the pickle file is not found.
        KeyError: If the key is not found in the dictionary.
        ValueError: If there is an error loading the pickle file.
        """
        filepath = os.path.join('..', 'pkl', 'threshold_dicts.pkl')
        try:
            with open(filepath, 'rb') as file:
                data = pickle.load(file)
            
            for item in data:
                if key in item:
                    return item[key]
            raise KeyError(f"Key '{key}' not found in the threshold dictionary.")
        except FileNotFoundError:
            raise FileNotFoundError(f"The file '{filepath}' was not found.")
        except pickle.PickleError:
            raise ValueError("Error occurred while loading the pickle file.")
        
    def get_price_features(self):
        """
        Calculates various price and volume features from historical data.

        Returns:
        pd.DataFrame: A DataFrame containing the calculated features.
        """
        aux = self.data.copy()
        df = pd.DataFrame(index=aux.index)
        df.index = pd.to_datetime(df.index).strftime('%Y-%m-%d')
        df['return_rate'] = aux['close'].pct_change()
        df['volume_change'] = aux['volume'].diff()
        df['volume_var'] = aux['volume'].pct_change() + 1
        df['price_range'] = aux['high'] - aux['low']
        df['price_var'] = df['price_range'] / aux['low']
        df['price_change'] = aux['close'] - aux['open']
        df['close_vol'] = aux['close'].expanding().std()
        df['low_vol'] = aux['low'].expanding().std()
        df['high_vol'] = aux['high'].expanding().std()
        df['open_vol'] = aux['open'].expanding().std()
        df['upper_wick'] = aux['high'] - aux[['open', 'close']].max(axis=1)
        df['lower_wick'] = aux[['open', 'close']].min(axis=1) - aux['low']
        df['wick_change'] = df['upper_wick'] - df['lower_wick']
        df['wick_var'] = df['wick_change'] / df['lower_wick']
        df['wick_vol'] = df['wick_change'].abs().expanding().std()
        df = df.apply(pd.to_numeric, errors='coerce')
        df = pd.concat([aux, df], axis=1)
        
        return df

    def get_candle_features(self, threshold_dict: dict = None):
        """
        Calculates various candlestick features based on price data and a threshold dictionary.

        Parameters:
        threshold_dict (dict): A dictionary containing thresholds for calculating features. If None, it loads the default dictionary.

        Returns:
        pd.DataFrame: A DataFrame containing the calculated candlestick features.
        """
        aux = self.get_price_features()
        if threshold_dict is None:
            threshold_dict = self.load_dict(key='threshold_dict')
        
        df = pd.DataFrame(index=aux.index)
        
        df[f'std_{threshold_dict["volatility_window"]}'] = aux['price_change'].rolling(window=threshold_dict['volatility_window']).std().abs()
        df['bearish_threshold'] = 1 + threshold_dict["bearish_threshold"] * df[f'std_{threshold_dict["volatility_window"]}'] 
        df['bullish_threshold'] = 1 + threshold_dict["bullish_threshold"] * df[f'std_{threshold_dict["volatility_window"]}'] 
        df['is_bearish'] = (aux['close'] <= (aux['open'] - df['bearish_threshold'])).astype(int)
        df['is_bullish'] = (aux['close'] >= (aux['open'] + df['bullish_threshold'])).astype(int)
        df['is_doji'] = (abs(aux['close'] - aux['open']) <= threshold_dict['doji_threshold']).astype(int)
        df['is_bearish_open_gap'] = (aux['close'] < aux['open'].shift(1)).astype(int)
        df['is_bullish_open_gap'] = (aux['open'] > aux['close'].shift(1)).astype(int)
        
        df = pd.concat([aux, df], axis=1)

        return df

    def fit_morning_star(self, threshold_dict: dict = None):
        """
        Identifies the morning star candlestick pattern in the historical data.

        Parameters:
        threshold_dict (dict): A dictionary containing thresholds for the morning star pattern. If None, it loads the default dictionary.

        Returns:
        pd.DataFrame: A DataFrame with a column indicating the presence of the morning star pattern.
        """
        df = self.get_candle_features()  
        if threshold_dict is None:
            threshold_dict = self.load_dict(key='morning_star_dict')
            display(pd.DataFrame([threshold_dict]))


        # Condition 1: Two days ago was a bearish candle and the close of that day is lower than the open of that day adjusted by the threshold
        condition1 = (df['is_bearish'].shift(2) == 1) & \
                    (df['close'].shift(2) + threshold_dict['bearish_threshold'] * df['price_change'].shift(2) <= df['open'].shift(2))

        # Condition 2: The previous day was a doji candle and had a bearish open gap
        condition2 = (df['is_bearish_open_gap'].shift(1) == 1) & (df['is_doji'].shift(1) == 1)

        # Condition 3: Today is a bullish candle and the price change from two days ago to today is significant
        condition3 = (df['is_bullish'] == 1) & \
                    (df['close'] - df['close'].shift(2) >= threshold_dict['bullish_threshold'] * df['price_change'].shift(2).abs())

        # Combine all conditions to determine the morning star pattern
        df['is_morning_star'] = ((condition1) & (condition2) & (condition3)).astype(int)
        
        return df
    
    def get_movings(self, 
                    threshold_dict: dict = None,
                    short_window=None,
                    long_window=None,
                    signal_window=None):
        
        df = self.get_candle_features()  
        if threshold_dict is None:
            threshold_dict = self.load_dict(key='rolling_cross_dict')
            display(pd.DataFrame([threshold_dict]))
            
        for key in threshold_dict:
            short = threshold_dict[key]['short']
            long = threshold_dict[key]['long']
            if 'sma' in key:
                df[f'sma_short_{short}']  =  df['close'].rolling(window=short, min_periods=1).mean()
                df[f'sma_long_{long}'] = df['close'].rolling(window=long, min_periods=1).mean()
                signal = np.where(df[f'sma_short_{short}'] > df[f'sma_long_{long}'], 1, np.where(df[f'sma_short_{short}'] < df[f'sma_long_{long}'], -1, 0))
                df[f'signal_sma_{short}_{long}'] = signal
                        
            elif 'ema' in key:
                    df[f'ema_short_{short}'] = df['close'].ewm(span=short, adjust=False).mean()
                    df[f'ema_Long_{long}'] = df['close'].ewm(span=long, adjust=False).mean()
                    signal = np.where(df[f'ema_short_{short}'] > df[f'ema_Long_{long}'], 1, np.where(df[f'ema_short_{short}'] < df[f'ema_Long_{long}'], -1, 0))
                    df[f'Signal_ema_{short}_{long}'] = signal

            # else: 
            #     'macd' in key:
            #     for signal_period in signal_window:
            #     df[f'ema_short_12'] = df['close'].ewm(span=12, adjust=False).mean()
            #     df[f'ema_Long_26'] = df['close'].ewm(span=26, adjust=False).mean()
            #     df['MACD'] = df[f'ema_short_12'] - df[f'ema_Long_26']
            #     df[f'Signal_Line_{signal_period}'] = df['MACD'].ewm(span=signal_period, adjust=False).mean()
            #     signal = np.where(df['MACD'] > df[f'Signal_Line_{signal_period}'], 1, np.where(df['MACD'] < df[f'Signal_Line_{signal_period}'], -1, 0))
            #     df[f'Signal_MACD_{signal_period}'] = signal
        
        return df
    

    def candlestick_chart(self, key: str ='is_morning_star', height: int = 900, offset: float = 20):
        """
        Identifies the morning star candlestick pattern in the historical data.

        Parameters:
        threshold_dict (dict): A dictionary containing thresholds for the morning star pattern. If None, it loads the default dictionary.

        Returns:
        pd.DataFrame: A DataFrame with a column indicating the presence of the morning star pattern.
        """
        if key == 'morning_star':
            aux = self.fit_morning_star()
        else:
            aux = self.get_candle_features()

        if key not in aux.columns:
            print(f"Key '{key}' not found in aux columns.")
            return None
        
        markers = aux[aux[key] == 1].copy()
        markers['close'] = markers['low'] - offset
        markers = markers.dropna(subset=['close'])
        
        trace = go.Candlestick(
            x=aux.index,
            open=aux["open"],
            high=aux["high"],
            low=aux["low"],
            close=aux["close"],
            name=self.ticker,
            yaxis="y"
        )
        
        marker_trace = go.Scatter(
            x=markers.index,
            y=markers['close'],
            mode='markers',
            marker=dict(
                color='blue',
                size=8,
                symbol='triangle-up'
            ),
            name=key,
            yaxis="y"
        )
        
        volume_colors = ['green' if aux['volume'][i] > aux['volume'][i-1] else 'red' for i in range(1, len(aux))]
        volume_colors.insert(0, 'green') 
        volume_trace = go.Bar(
            x=aux.index,
            y=aux['volume'],
            marker_color=volume_colors,
            name='Volume',
            yaxis="y2"
        )
        layout = go.Layout(
            title=f"{self.ticker} Candlestick Chart markers: {key.capitalize()}",
            xaxis=dict(title="Date"),
            yaxis=dict(title="Price", domain=[0.3, 1]),
            yaxis2=dict(title="Volume", domain=[0, 0.2]),
            height=height,
            barmode='relative'
        )
        
        fig = go.Figure(data=[trace, marker_trace, volume_trace], layout=layout)
        return fig


In [30]:

ticker = CandleFit('AAPL')
df = ticker.get_movings()




,sma_5_l0,sma_5_20,sma_10_30,sma_10_50,sma_20_50,sma_20_100,sma_50_100,sma_50_150,sma_100_200,ema_5_10,ema_5_20,ema_10_30,ema_10_50,ema_20_50,ema_20_100,ema_50_100,ema_50_150,ema_100_200,macd_9_12,macd_9_30,macd_12_26,macd_12_50,macd_26_50,macd_26_100,macd_50_100,macd_50_200
0,"{'short': 5, 'long': 10}","{'short': 5, 'long': 20}","{'short': 10, 'long': 30}","{'short': 10, 'long': 50}","{'short': 20, 'long': 50}","{'short': 20, 'long': 100}","{'short': 50, 'long': 100}","{'short': 50, 'long': 150}","{'short': 100, 'long': 200}","{'short': 5, 'long': 10}","{'short': 5, 'long': 20}","{'short': 10, 'long': 30}","{'short': 10, 'long': 50}","{'short': 20, 'long': 50}","{'short': 20, 'long': 100}","{'short': 50, 'long': 100}","{'short': 50, 'long': 150}","{'short': 100, 'long': 200}","{'short': 12, 'long': 26}","{'short': 30, 'long': 60}","{'short': 12, 'long': 26}","{'short': 12, 'long': 50}","{'short': 26, 'long': 50}","{'short': 26, 'long': 100}","{'short': 50, 'long': 100}","{'short': 50, 'long': 200}"


In [31]:
d

,open,high,low,close,volume,dividends,stock splits,return_rate,volume_change,volume_var,price_range,price_var,price_change,close_vol,low_vol,high_vol,open_vol,upper_wick,lower_wick,wick_change,wick_var,wick_vol,std_7,bearish_threshold,bullish_threshold,is_bearish,is_bullish,is_doji,is_bearish_open_gap,is_bullish_open_gap,sma_short_5,sma_long_10,signal_sma_5_10,sma_long_20,signal_sma_5_20,sma_short_10,sma_long_30,signal_sma_10_30,sma_long_50,signal_sma_10_50,sma_short_20,signal_sma_20_50,sma_long_100,signal_sma_20_100,sma_short_50,signal_sma_50_100,sma_long_150,signal_sma_50_150,sma_short_100,sma_long_200,signal_sma_100_200,ema_short_5,ema_Long_10,Signal_ema_5_10,ema_Long_20,Signal_ema_5_20,ema_short_10,ema_Long_30,Signal_ema_10_30,ema_Long_50,Signal_ema_10_50,ema_short_20,Signal_ema_20_50,ema_Long_100,Signal_ema_20_100,ema_short_50,Signal_ema_50_100,ema_Long_150,Signal_ema_50_150,ema_short_100,ema_Long_200,Signal_ema_100_200
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-07-31,52.216582,53.410889,50.981259,51.401073,277125600,0.0,0.0,NaN,NaN,NaN,2.429630,0.047657,-0.815508,NaN,NaN,NaN,NaN,1.194307,0.419815,0.774492,1.844842,NaN,NaN,NaN,NaN,0,0,0,0,0,51.401073,51.401073,0,51.401073,0,51.401073,51.401073,0,51.401073,0,51.401073,0,51.401073,0,51.401073,0,51.401073,0,51.401073,51.401073,0,51.401073,51.401073,0,51.401073,0,51.401073,51.401073,0,51.401073,0,51.401073,0,51.401073,0,51.401073,0,51.401073,0,51.401073,51.401073,0
2019-08-01,51.608570,52.605034,49.881049,50.288799,216071600,0.0,0.0,-0.021639,-61054000.0,0.779688,2.723985,0.054610,-1.319771,0.786497,0.777966,0.569825,0.429929,0.996464,0.407750,0.588714,1.443808,0.131365,NaN,NaN,NaN,0,0,0,1,1,50.844936,50.844936,0,50.844936,0,50.844936,50.844936,0,50.844936,0,50.844936,0,50.844936,0,50.844936,0,50.844936,0,50.844936,50.844936,0,51.030315,51.198842,-1,51.295143,-1,51.198842,51.329314,-1,51.357455,-1,51.295143,-1,51.379048,-1,51.357455,-1,51.386341,-1,51.379048,51.390006,-1
2019-08-02,49.589112,49.806257,48.648143,49.224789,163448400,0.0,0.0,-0.021158,-52623200.0,0.756455,1.158114,0.023806,-0.364323,1.088232,1.167186,1.891909,1.375469,0.217145,0.576645,-0.359500,-0.623433,0.207875,NaN,NaN,NaN,0,0,0,1,0,50.304887,50.304887,0,50.304887,0,50.304887,50.304887,0,50.304887,0,50.304887,0,50.304887,0,50.304887,0,50.304887,0,50.304887,50.304887,0,50.428473,50.839923,-1,51.097966,-1,50.839923,51.193538,-1,51.273821,-1,51.097966,-1,51.336390,-1,51.273821,-1,51.357712,-1,51.336390,51.368462,-1
2019-08-05,47.769900,47.929138,46.464605,46.647972,209572000,0.0,0.0,-0.052348,46123600.0,1.282191,1.464533,0.031519,-1.121928,2.032918,1.936794,2.531684,2.024216,0.159238,0.183367,-0.024129,-0.131588,0.323206,NaN,NaN,NaN,0,0,0,1,0,49.390658,49.390658,0,49.390658,0,49.390658,49.390658,0,49.390658,0,49.390658,0,49.390658,0,49.390658,0,49.390658,0,49.390658,49.390658,0,49.168306,50.077750,-1,50.674157,-1,50.077750,50.900276,-1,51.092415,-1,50.674157,-1,51.243550,-1,51.092415,-1,51.295331,-1,51.243550,51.321492,-1
2019-08-06,47.364557,47.789202,46.816863,47.531036,143299200,0.0,0.0,0.018930,-66272800.0,0.683771,0.972339,0.020769,0.166480,1.947102,1.939370,2.605732,2.189020,0.258165,0.547694,-0.289528,-0.528632,0.287540,NaN,NaN,NaN,0,0,0,1,1,49.018734,49.018734,0,49.018734,0,49.018734,49.018734,0,49.018734,0,49.018734,0,49.018734,0,49.018734,0,49.018734,0,49.018734,49.018734,0,48.622550,49.614711,-1,50.374812,-1,49.614711,50.682905,-1,50.952753,-1,50.374812,-1,51.170035,-1,50.952753,-1,51.245473,-1,51.170035,51.283776,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-24,224.000000,224.800003,217.130005,218.539993,61777600,0.0,0.0,-0.028754,21817300.0,1.545974,7.669998,0.035324,-5.460007,42.311308,42.017272,42.611129,42

In [ ]:
df[df['is_bullish']==1].head(2)


In [ ]:
df[df.index == '2019-08-12']

In [ ]:
50.610653 - 48.554455 

In [ ]:
df[df.index == '2019-08-09']

In [ ]:
48.677971 - 48.75305